安装opencompass：Kaggle上已经为我们准备好了其他常用包，只需安装opencompass用于评测即可。如果不在Kaggle上运行，则还需要安装其他必要包。

In [7]:
!pip install "opencompass[full]"
# !pip install pytorch transformers datasets "opencompass[full]"

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 8.7 MB/s eta 0:00:0000:0100:01m
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 2.5 MB/s eta 0:00:0000:0100:02
  

In [1]:
import wandb
wandb.login(key="xxx")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: char-lotte (char-lotte-xia). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/xiaxinyuan/.netrc


True

# 指令微调

In [2]:
"""
The main program for finetuning LLMs with Huggingface Transformers Library.

ALL SECTIONS WHERE CODE POSSIBLY NEEDS TO BE FILLED IN ARE MARKED AS TODO.
"""

import argparse
from dataclasses import dataclass, field
from typing import Optional, List, Dict
import sys
import torch
from transformers import TrainingArguments, HfArgumentParser, Trainer, AutoTokenizer, AutoModelForCausalLM
import datasets

/home/xiaxinyuan/.conda/envs/dino/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Define the arguments required for the main program.
# NOTE: You can customize any arguments you need to pass in.
# @dataclass
class ModelArguments:
    """Arguments for model
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the LLM to fine-tune or its name on the Hugging Face Hub."
        }
    )
    torch_dtype: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype."
            ),
            "choices": ["bfloat16", "float16", "float32"],
        },
    )
    # TODO: add your model arguments here
    pass


@dataclass
class DataArguments:
    """Arguments for data
    """
    dataset_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the fine-tuning dataset or its name on the Hugging Face Hub."
        }
    )
    # TODO: add your data arguments here
    max_length: int = field(
        default=512,
        metadata={
            "help": "The max length of tokenized data."
        }
    )

In [5]:
# The main function
# NOTE You can customize some logs to monitor your program.
def finetune():
    # TODO Step 1: Define an arguments parser and parse the arguments
    # NOTE Three parts: model arguments, data arguments, and training arguments
    # HINT: Refer to 
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/internal/trainer_utils#transformers.HfArgumentParser
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/trainer#transformers.TrainingArguments
    # parser = ...
    # model_args, data_args, training_args = ...
    ### cjy
    parser = HfArgumentParser((ModelArguments, DataArguments, TrainingArguments)) 
    """
    hugging face argument parser
    """
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    """
    return tuple
    DataClass 实例的顺序与传递给 initializer.abspath 的顺序相同
    如果适用，将更多（非 DataClass 支持的）参数的附加命名空间添加到解析器 初始化后。
    剩余参数字符串的可能列表。（与 argparse 相同。ArgumentParser.parse_known_args）
    """
    ### cjy

    # TODO Step 2: Load tokenizer and model
    # HINT 1: Refer to
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/tokenizer#tokenizer
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/qwen2
    # HINT 2: To save training GPU memory, you need to set the model's parameter precision to half-precision (float16 or bfloat16).
    #         You may also check other strategies to save the memory!
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/llama2#usage-tips
    #   * https://huggingface.co/docs/transformers/perf_train_gpu_one
    #   * https://www.53ai.com/news/qianyanjishu/2024052494875.html
    # tokenizer = ...
    # model = ...
    ## cjy
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_args.model_name_or_path)
    """
    从预训练模型的 tokenizer 配置中加载 tokenizer 的配置
    input:
        pretrained_model_name_or_path: 预训练模型的名称或路径（tokenizer 配置文件 所在的目录 即“Qwen2.5-0.5B”）
    功能:
        1 函数首先尝试从指定的模型名称或路径加载 tokenizer 配置文件
        2 如果找不到配置文件，则从模型库中加载默认的 tokenizer 配置
    """
    model = AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path=model_args.model_name_or_path, 
        torch_dtype=model_args.torch_dtype,
        trust_remote_code=True,  # Qwen模型需要这个参数
        device_map="auto"  # 可选，用于自动处理模型加载到设备
    )
    """
    用于从预训练模型加载模型实例。 
    """
    ### cjy

    # TODO Step 3: Load dataset
    # HINT: https://huggingface.co/docs/datasets/v3.1.0/en/package_reference/main_classes#datasets.Dataset
    # dataset = ...
    ### cjy
    dataset = datasets.load_dataset(path='csv', data_files=data_args.dataset_path)
    ### cjy

    # TODO Step 4: Define the data collator function
    # NOTE During training, for each model parameter update, we fetch a batch of data, perform a forward and backward pass,
    # and then update the model parameters. The role of the data collator is to process the data (e.g., padding the data within
    # a batch to the same length) and format the batch into the input required by the model.
    # 数据整理器（data collator）函数。这个函数的主要任务是处理从数据集中加载的每个数据批次，使其符合模型所需的输入格式
    # In this assignment, the purpose of the custom data_collator is to process each batch of data from the dataset loaded in
    # Step 3 into the format required by the model. This includes tasks such as tokenizing the data, converting each token into 
    # an ID sequence, applying padding, and preparing labels.
    # 对数据进行标记化（tokenization）。将每个标记转换为 ID 序列。填充（padding），确保批次中的所有数据具有相同的长度。准备标签（labels），以便在训练过程中使用
    # HINT:
    #   * Before implementation, you should:
    #      1. Clearly understand the format of each sample in the dataset loaded in Step 3.
    #      2. Understand the input format required by the model (https://huggingface.co/docs/transformers/model_doc/qwen2#transformers.Qwen2ForCausalLM).
    #         Reading its source code also helps!

    def data_collator(batch: List[Dict]):
        """
        batch: list of dict, each dict of the list is a sample in the dataset.
        """
        # pass
        ### cjy
        # 期望字典输出
        new_batch = {
            "input_ids": [], #  (torch.LongTensor of shape (batch_size, sequence_length)) 
            # input ids: 通常是作为输入传递给模型的唯一必需参数. token(模型输入)的数字化表示. https://huggingface.co/docs/transformers/glossary#input-ids
            # 用 PreTrainedTokenizer.encode() 方法得到
            "labels": [] #  (torch.LongTensor of shape (batch_size, sequence_length))
            # labels: 标签是模型训练的目标值，用于计算损失函数。 (loss, optional, returned when labels is provided))
            # 用 PreTrainedTokenizer.encode() 方法得到 
        }
        for sample in batch:
            new_batch["input_ids"].append(tokenizer.encode("instruction:"+sample["instruction"] if sample["instruction"] else "" + "\n input:"+ sample["input"] if sample["input"] else "", padding='max_length', max_length=data_args.max_length, truncation=True))
            # padding,max_length,truncation: 用于处理输入序列长度不一致的问题。
            new_batch["labels"].append(tokenizer.encode(sample["output"],padding='max_length',  max_length=data_args.max_length, truncation=True))
        new_batch["input_ids"] = torch.tensor(new_batch["input_ids"])
        new_batch["labels"] = torch.tensor(new_batch["labels"])
        return new_batch
        ### cjy

    # TODO Step 5: Define the Trainer
    # HINT: https://huggingface.co/docs/transformers/main_classes/trainer
    trainer = Trainer(
        # ...,
        # model=model,
        ### cjy
        model=model, # 模型  (PreTrainedModel or torch.nn.Module)
        args=training_args, # 训练参数 (TrainingArguments)
        data_collator=data_collator, # 数据整理器 (DataCollator)
        train_dataset=dataset['train'], # 训练数据集 (Dataset)
        ### cjy
    )

    # Step 6: Train!
    trainer.train()

In [6]:
# Pass your training arguments.
# NOTE [IMPORTANT!!!] DO NOT FORGET TO PASS PROPER ARGUMENTS TO SAVE YOUR CHECKPOINTS!!!
sys.argv = [
    "notebook", 
    # "--arg1", "value1",
    # "--arg2", "value2",
    # ...
    ### cjy
    "--model_name_or_path", "/kaggle/input/qwen2.5/transformers/0.5b/1",
    "--dataset_path", "/kaggle/input/alpaca-language-instruction-training/train.csv",
    "--torch_dtype", "bfloat16", #see Qwen2.5-0.5B/config.json?
    "--output_dir", "/kaggle/working/output/", # --output_dir 参数在 TrainingArguments 中有
    "--remove_unused_columns", "False", #ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [output, instruction, input]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`
    "--max_length", "256",
    ### cjy
    "--per_device_train_batch_size", "8",  # 设置训练的 batch size
    "--per_device_eval_batch_size", "8", 
    "--save_steps", "1000",
    "--save_total_limit", "4",
]
finetune()


TypeError: must be called with a dataclass type or instance

# 评测模型

In [7]:
PLM_MODEL_PATH = "/kaggle/input/qwen2.5/transformers/0.5b/1"
SFT_MODEL_PATH = "/kaggle/input/qwen_checkpoint_14000/transformers/default/1/checkpoint_14000"

如果你有多个GPU，可以修改下面的--hf-num-gpus参数来加速评测。

In [8]:
# !opencompass \
#     --datasets mmlu_ppl hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
#     --summarizer example \
#     --hf-type base \
#     --hf-path {PLM_MODEL_PATH} \
#     --tokenizer-kwargs padding_side="left" truncation="left" \
#     --max-seq-len 2048 \
#     --batch-size 4 \
#     --hf-num-gpus 2 \
#     --work-dir "/kaggle/working/evals/plm" \
#     --debug

In [9]:
!cd /
!opencompass \
    --datasets mmlu_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path {SFT_MODEL_PATH} \
    --tokenizer-kwargs padding_side="left" truncation="left" \
    --max-seq-len 2048 \
    --batch-size 4 \
    --hf-num-gpus 2 \
    --work-dir "/kaggle/working/evals/sft" \
    --debug
    

/opt/conda/lib/python3.10/site-packages/opencompass/__init__.py:19: UserWarning: Starting from v0.4.0, all AMOTIC configuration files currently located in `./configs/datasets`, `./configs/models`, and `./configs/summarizers` will be migrated to the `opencompass/configs/` package. Please update your configuration file paths accordingly.
  _warn_about_config_migration()
12/18 03:27:09 - OpenCompass - INFO - Loading mmlu_ppl: /opt/conda/lib/python3.10/site-packages/opencompass/configs/./datasets/mmlu/mmlu_ppl.py
12/18 03:27:09 - OpenCompass - INFO - Loading example: /opt/conda/lib/python3.10/site-packages/opencompass/configs/./summarizers/example.py
12/18 03:27:09 - OpenCompass - INFO - Current exp folder: /kaggle/working/evals/sft/20241218_032709
12/18 03:27:09 - OpenCompass - WARNING - SlurmRunner is not used, so the partition argument is ignored.
12/18 03:27:09 - OpenCompass - INFO - ./data/mmlu/ does not exist!Start Download data automatically!If you have downloaded the data before,Yo

In [2]:
import kagglehub

# Download latest version
path = kagglehub.model_download("qwen-lm/qwen2.5/transformers/3b-instruct")

print("Path to model files:", path)

100%|██████████| 242/242 [00:00<00:00, 284kB/s]


100%|██████████| 7.21k/7.21k [00:00<00:00, 25.9MB/s]



100%|██████████| 1.48k/1.48k [00:00<00:00, 6.03MB/s]


100%|██████████| 661/661 [00:00<00:00, 1.08MB/s]



100%|██████████| 1.59M/1.59M [00:01<00:00, 1.00MB/s]


100%|██████████| 7.13k/7.13k [00:00<00:00, 19.5MB/s]


100%|██████████| 4.81k/4.81k [00:00<00:00, 12.5MB/s]

100%|██████████| 34.7k/34.7k [00:00<00:00, 180kB/s]




































































100%|██████████| 2.65M/2.65M [00:04<00:00, 685kB/s]































100%|██████████| 6.71M/6.71M [00:06<00:00, 1.04MB/s]









































































































































































































































































































































































































































































































































































































































































































































































































































Path to model files: /home/xiaxinyuan/.cache/kagglehub/models/qwen-lm/qwen2.5/transformers/3b-instruct/1
